In [1]:
from aicsimageio import AICSImage, imread, imread_dask
from aicspylibczi import CziFile
import numpy as np
import napari
import numpy as np

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import morphology
from skimage import io, measure, segmentation
from skimage import exposure
from skimage.exposure import rescale_intensity
from skimage.morphology import watershed, dilation
from skimage.morphology import white_tophat, black_tophat, disk, square, ball, closing, square
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.feature import peak_local_max
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu, threshold_triangle, rank, median, gaussian, scharr
from skimage.segmentation import clear_border
from skimage.segmentation import random_walker
from skimage.color import label2rgb
from skimage.util import invert

from scipy.ndimage import distance_transform_edt
from scipy import ndimage as ndi


In [2]:
viewer = napari.Viewer()
spacing = np.array([1.0, 1.0, 1.0])

[WARNING: _qt: 187 2021-07-09 19:46:22,929] QWindowsWindow::setGeometry: Unable to set geometry 1090x677-1687+1114 (frame: 1108x724-1696+1076) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1365x848-1686+1121 (frame: 1383x895-1695+1083) margins: 9, 38, 9, 9 minimum size: 608x580 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=626,627 maxtrack=0,0)


In [3]:
filename = "D:\Testdata_Zeiss\Mitochondria_EM_with_DNN\original_data\mitochondria_train_01_seg_ov_small.czi"

czi = CziFile(filename)

# get number of zplanes
dimensions = czi.get_dims_shape()
print(dimensions)
print(czi.dims)
print(czi.size)
numCH = dimensions[0]['C'][1]
numZ = dimensions[0]['Z'][1]


[{'X': (0, 512), 'Y': (0, 512), 'Z': (0, 60), 'C': (0, 2), 'T': (0, 1), 'M': (0, 4)}]
TCZMYX
(1, 2, 60, 4, 512, 512)


In [4]:
img3d = np.empty((dimensions[0]['Z'][1], dimensions[0]['C'][1], dimensions[0]['X'][1], dimensions[0]['X'][1]))

for ch in range(numCH):
    for z in range(numZ):

        plane2d = czi.read_mosaic(T=0, C=ch, Z=z, scale_factor=1)
        img3d[z, ch, ...] = np.squeeze(plane2d)
    
print(img3d.shape)

for ch in range(numCH):
    viewer.add_image(img3d[:, ch, ...], name=str(ch), scale=spacing, blending='additive')

(60, 2, 512, 512)


In [5]:
edges = scharr(img3d[:, 1, ...])
labels = measure.label(img3d[:, 1, ...])
print(labels.shape)
viewer.add_image(labels, name="labels", scale=spacing, blending='additive')

(60, 512, 512)


<Image layer 'labels' at 0x19dd8542f40>

In [6]:
transformed1 = ndi.distance_transform_edt(labels, sampling=spacing)
viewer.add_image(transformed1, name="transformed1", scale=spacing, blending='additive')

<Image layer 'transformed1' at 0x19de7474be0>

In [7]:
transformed2 = transformed1 >= transformed1.max() * 0.4
viewer.add_image(transformed2, name="transformed2", scale=spacing, blending='additive')

<Image layer 'transformed2' at 0x19dbdc0a6d0>

In [8]:
maxima1 = morphology.local_maxima(transformed1)
#maxima2 = morphology.local_maxima(transformed2)
viewer.add_image(maxima1, name="maxima1", scale=spacing, blending='additive')
#viewer.add_image(maxima2, name="maxima2", scale=spacing, blending='additive')

<Image layer 'maxima1' at 0x19d87828040>

In [9]:
viewer.add_points(
    np.transpose(np.nonzero(maxima1)),
    name='bad points1',
    scale=spacing,
    size=4,
    n_dimensional=True,  # points have 3D "extent"
)

<Points layer 'bad points1' at 0x19de7474eb0>

In [15]:
markers_big = morphology.dilation(maxima1, morphology.ball(5))
segmented = segmentation.watershed(edges, markers_big, mask=labels)
segmented_padded = np.pad(
    segmented,
    ((1, 1), (0, 0), (0, 0)),
    mode='constant',
    constant_values=0,
)
interior_labels = segmentation.clear_border(segmented_padded)[1:-1]

In [16]:
viewer.add_image(interior_labels, name="segmented", scale=spacing, blending='additive')

<Image layer 'segmented' at 0x19d8fb8e460>

In [14]:
regionprops = measure.regionprops(segmented, intensity_image=img3d[:, 0, ...])

supported = [] 
unsupported = []

for prop in regionprops[0]:
    try:
        regionprops[0][prop]
        supported.append(prop)
    except NotImplementedError:
        unsupported.append(prop)

print("Supported properties:")
print("  " + "\n  ".join(supported))
print()
print("Unsupported properties:")
print("  " + "\n  ".join(unsupported))

MemoryError: Unable to allocate 884. GiB for an array with shape (118632961450,) and data type float64

In [ ]:
import pandas as pd


info_table = pd.DataFrame(
    measure.regionprops_table(
        interior_labels,
        intensity_image=nuclei,
        properties=['label', 'slice', 'area', 'mean_intensity', 'solidity'],
    )
).set_index('label')